<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/llama_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install accelerate
!pip install -q git+https://github.com/huggingface/transformers.git
#!pip install -q git+https://github.com/zphang/transformers@StellaAthena-patch-1

In [4]:
from transformers import LlamaTokenizer, LlamaForCausalLM
load_in_8bit_fp32_cpu_offload=True

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf",
    #load_in_8bit=True,
    device_map='auto',

)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
tokenizer.vocab_size

32000

In [6]:
text = "شب چهارم دیگه طاقت نیاوردم یه فریدون فروغی زدمو وسطای کورش یغمایی بودم"
output = tokenizer.encode(text)
#output = tokenizer.encode("اوردم")
print(output)
print([tokenizer.decode([ i ]) for i in output])

[1, 29871, 30256, 30177, 29871, 30905, 30204, 30112, 30156, 30159, 29871, 30172, 30202, 30421, 30204, 29871, 30367, 30112, 30265, 30195, 29871, 30162, 30202, 30112, 30171, 30156, 30172, 30159, 29871, 30202, 30204, 29871, 30241, 30156, 30202, 30172, 30171, 30162, 29871, 30241, 30156, 30171, 30611, 30202, 29871, 30295, 30172, 30159, 30171, 29871, 30171, 30198, 30367, 30112, 30202, 29871, 30343, 30171, 30156, 30256, 29871, 30202, 30611, 30159, 30112, 30202, 30202, 29871, 30177, 30171, 30172, 30159]
[' ', ' ', ' ش', ' ب', ' ', ' چ', ' ه', ' ا', ' ر', ' م', ' ', ' د', ' ی', ' گ', ' ه', ' ', ' ط', ' ا', ' ق', ' ت', ' ', ' ن', ' ی', ' ا', ' و', ' ر', ' د', ' م', ' ', ' ی', ' ه', ' ', ' ف', ' ر', ' ی', ' د', ' و', ' ن', ' ', ' ف', ' ر', ' و', ' غ', ' ی', ' ', ' ز', ' د', ' م', ' و', ' ', ' و', ' س', ' ط', ' ا', ' ی', ' ', ' ک', ' و', ' ر', ' ش', ' ', ' ی', ' غ', ' م', ' ا', ' ی', ' ی', ' ', ' ب', ' و', ' د', ' م']


In [8]:
import pandas as pd
import torch
a = torch.squeeze(model.get_input_embeddings().weight.clone()).detach().numpy()
px = pd.DataFrame(index=range(tokenizer.vocab_size) )

def convert_ids_to_tokens(i):
    outputtokens = tokenizer.convert_ids_to_tokens(i)
    #return decoder.decode(outputtokens)
    return outputtokens


px['token'] = px.index.to_series().apply(convert_ids_to_tokens, convert_dtype=True, args=())
px

,token
0,<unk>
1,<s>
2,</s>
3,<0x00>
4,<0x01>
...,...
31995,黃
31996,왕
31997,收
31998,弘


In [15]:
embtable = torch.squeeze(model.get_input_embeddings().weight.clone())


In [40]:
import gc

del model

gc.collect()

9321

In [39]:
px.to_csv('/content/drive/MyDrive/DataSet/vocab7b.csv')

In [38]:
torch.save(embtable, '/content/drive/MyDrive/DataSet/embeding_tensor_7b_llama.pt')

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
import numpy as np

text_indexbase = tokenizer.encode('fielder')
print(text_indexbase)
text_indexbase = 12911
embdbase = embtable[text_indexbase].detach().numpy()
print(embdbase.shape)
max=0


def calc_similarity(i):
    embd = embtable[i].detach().numpy()
    smltry = (np.dot(embd,embdbase)).item()
    return smltry

px['similarity'] = px.index.to_series().apply(calc_similarity, convert_dtype=True, args=())

px.sort_values(['similarity'], ascending=False).head(20)

[1, 29607, 672]
(4096,)


,token,similarity
12911,Visible,1.724609
7962,▁visible,0.734375
12872,visible,0.623535
26401,▁visibility,0.432617
23318,Visibility,0.416504
28814,visibility,0.390625
10861,Enabled,0.383789
8592,Selected,0.378174
27597,▁invisible,0.373535
20215,▁Observable,0.324219
